OSIsoft Cloud Services Quantities and units (Units of Measure)

Requirements:

1. Juptyer notebook or lab
2. Python
3. ocs_sample_library https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/library_samples/Python3 
<p>Note: this notebook uses a customized version of the Python library, some of these features are not yet in the github library.
4. config.ini configured for your OSIsoft Cloud Services environment https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/basic_samples/SDS/Python/SDSPy/Python3
5. Familiarity with namespaces, types and streams and for more information see https://cloud.osisoft.com

Objectives:

Create a type with a unit definition and a stream based on that type
Create a stream that overrides the unit definition of the type
Query streams with and without a property override for the defined units

Using the OSIsoft Cloud Services Portal > API Console is one way to view quantities and units<br>
For example - for a specific quantity and unit:<br>
/Namespaces/dataset/Quantities/Temperature<br>
/Namespaces/dataset/Quantities/Temperature/units/degree Fahrenheit<br>

In [1]:
# specify a unique prefix to create objects
example_prefix = 'SDSUom.'

In [2]:
# setup the environment

import configparser
import logging
# For general usage when package is installed and not using customized features
#from ocs_sample_library_preview import *
# For testing/running modified github version
from OSI_Samples.ocs_samples.library_samples.Python3.ocs_sample_library_preview import *
import json
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import pandas as pd
import logging

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# Read the configuration informatation for your OSIsoft Cloud Services acccount from config.ini

config = configparser.ConfigParser()
config.read('config.ini')

ocsClient = OCSClient(config.get('Access', 'ApiVersion'), config.get('Access', 'Tenant'), config.get('Access', 'Resource'), 
                        config.get('Credentials', 'ClientId'), config.get('Credentials', 'ClientSecret'))
        
namespace_id = config.get('Configurations', 'Namespace')

In [3]:
# Create a type with a specified UoM for temperature

thetype = {'sdstypecode': SdsTypeCode.Object, 'id': f'{example_prefix}bearing_temperature',
            'name': f'{example_prefix}bearing_temperature', 'description': 'bearing temperature'}

sds_type_properties = []

#sds type code
sds_type_code = SdsType()
sds_type_code.Id = "DateTime"
sds_type_code.SdsTypeCode = SdsTypeCode.DateTime

# sds type property

sds_data_type = SdsTypeProperty()
sds_data_type.Id = "timestamp"
sds_data_type.SdsType = sds_type_code
sds_data_type.IsKey = True
sds_type_properties.append(sds_data_type)

# sds type code
sds_type_code = SdsType()
sds_type_code.Id = "Double"
sds_type_code.SdsTypeCode = SdsTypeCode.Double

# sds type property

sds_data_type =  SdsTypeProperty()
sds_data_type.Id = "temperature"
sds_data_type.SdsType = SdsType.fromJson({"SdsTypeCode" : SdsTypeCode.Double.value })
sds_data_type.Uom = "degree Fahrenheit"
sds_type_properties.append(sds_data_type)

# SDS type definition

sdstype = SdsType()
sdstype.Id = thetype['id']
sdstype.SdsTypeCode = thetype['sdstypecode']
sdstype.Name = thetype['name']
sdstype.Description=thetype['description']
sdstype.Properties = sds_type_properties

ocsClient.acceptverbosity = False
try:
    new_type = ocsClient.Types.getOrCreateType(namespace_id, sdstype)
except:
    new_type = ocsClient.Types.getType(namespace_id,sdstype.Id)
    print("Type already exists, querying existting type, verify object is defined as expected!")
print(json.dumps(json.loads(new_type.toJson()),indent=4))

{
    "SdsTypeCode": 1,
    "Properties": [
        {
            "Id": "timestamp",
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 16,
                "Id": "DateTime",
                "Name": null,
                "Description": null,
                "ExtrapolationMode": null,
                "InterpolationMode": null
            },
            "Value": null,
            "Order": null,
            "IsKey": true,
            "InterpolationMode": null,
            "Uom": null
        },
        {
            "Id": "temperature",
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 14,
                "Id": null,
                "Name": null,
                "Description": null,
                "ExtrapolationMode": null,
                "InterpolationMode": null
            },
            "Value": null,
            "Order": null,
            "IsKey": false,
            "InterpolationMode": null,
            "U

In [4]:
# Create a stream using the created type
stream_name = 'Asset3_bearing3'
new_stream = SdsStream(id=f'{example_prefix}{stream_name}',name=f'bearing3 temperature1', description=f'Bearing 3 temperature sensor1', typeId=f'{example_prefix}bearing_temperature')
stream = ocsClient.Streams.getOrCreateStream(namespace_id,new_stream)
print(f'id: {stream.Id}, name: {stream.Name}')

id: SDSUom.Asset3_bearing3, name: bearing3 temperature1


In [5]:
# Add stream events
ocsClient.Streams.updateValues(namespace_id,stream.Id,json.dumps([{'Timestamp': '2019-02-11T00:00:00Z', 'Temperature': 100},
 {'Timestamp': '2019-02-11T00:00:01Z', 'Temperature': 35.492},
 {'Timestamp': '2019-02-11T00:00:02Z', 'Temperature': 35.469},
 {'Timestamp': '2019-02-11T00:00:03Z', 'Temperature': 35.422},
 {'Timestamp': '2019-02-11T00:00:04Z', 'Temperature': 35.414},
 {'Timestamp': '2019-02-11T00:00:05Z', 'Temperature': 35.32},
 {'Timestamp': '2019-02-11T00:00:06Z', 'Temperature': 35.227},
 {'Timestamp': '2019-02-11T00:00:07Z', 'Temperature': 35.242},
 {'Timestamp': '2019-02-11T00:00:08Z', 'Temperature': 35.16},
 {'Timestamp': '2019-02-11T00:00:09Z', 'Temperature': 35.176}]))

From the OCS Portal API Console request conversion to celsius using a POST request with the following JSON as the body:<br>
Note: substitute DerekETesting with your namespace id
```
POST /Namespaces/DerekETesting/Streams/SDSUom.Asset3_bearing3/Data/Transform/First

[
  {
    "SdsTypePropertyId" : "temperature",
    "Uom" : "degree Celsius" 
  }
]
```

Query the stream, using a property override to request the result in celsius, instead of the type defined Farenheit<br>
![alt text](images/sds_uom_post_celsius.png "OCS Portal - API Console POST query")

From the OCS Portal API Console create a stream with an override to define the temperature property as celsius, instead of the type defined fahrenheit using the URI and POST body
Note: substitute DerekETesting with your namespace id

```
POST /Namespaces/DerekETesting/Streams/SdsUom.Asset3_bearing4

{
	"TypeId": "SDSUom.bearing_temperature",
	"Id": "SdsUom.Asset3_bearing4",
	"Name": "bearing4 temperature1",
	"Description": "Bearing 4 temperature sensor1",
	"PropertyOverrides": [
  {
    "SdsTypePropertyId" : "temperature",
    "Uom" : "degree Celsius" 
  }
]
}
```

In [6]:
# Add stream events
logger.setLevel(logging.CRITICAL)
logging.debug("tap")
try:
    ocsClient.Streams.updateValues(namespace_id,'SDSUom.Asset3_bearing4',json.dumps([{'Timestamp': '2019-02-11T00:00:00Z', 'Temperature': 100},
     {'Timestamp': '2019-02-11T00:00:01Z', 'Temperature': 35.492},
     {'Timestamp': '2019-02-11T00:00:02Z', 'Temperature': 35.469},
     {'Timestamp': '2019-02-11T00:00:03Z', 'Temperature': 35.422},
     {'Timestamp': '2019-02-11T00:00:04Z', 'Temperature': 35.414},
     {'Timestamp': '2019-02-11T00:00:05Z', 'Temperature': 35.32},
     {'Timestamp': '2019-02-11T00:00:06Z', 'Temperature': 35.227},
     {'Timestamp': '2019-02-11T00:00:07Z', 'Temperature': 35.242},
     {'Timestamp': '2019-02-11T00:00:08Z', 'Temperature': 35.16},
     {'Timestamp': '2019-02-11T00:00:09Z', 'Temperature': 35.176}]))
except Exception as e:
    print(str(e))
    print("Error may be due to missing stream, see earlier cell for how to create stream in OCS API Console")

In [7]:
# Query the first stored value
ocsClient.Streams.getFirstValue(namespace_id,'SDSUom.Asset3_bearing4',None)

{'timestamp': '2019-02-11T00:00:00Z', 'temperature': 100.0}

Query the stream, using a property override to request the result in Fahrenheit, instead of the stream defined Celsius
![alt text](files/sds_uom_post.png "OCS Portal - API Console POST query")

Example from previous screenshot, request stream event in Fahrenheit using OCS Portal. URI & POST body:

```
POST /Namespaces/DerekETesting/Streams/SDSUom.Asset3_bearing4/Data/Transform/First
[
  {
    "SdsTypePropertyId" : "temperature",
    "Uom" : "degree Fahrenheit" 
  }
]
```

In [8]:
# Clean-up environment

def cleanup(run=False):
        for stream in ocsClient.Streams.getStreams(namespace_id,f'{example_prefix}*'):
            print(f'stream id: {stream.Id}, stream name: {stream.Name}')
            if run:
                try:
                    ocsClient.Streams.deleteStream(namespace_id,stream.Id)
                    ocsClient.Types.deleteType(namespace_id,stream.TypeId)
                except:
                    pass
        for type in ocsClient.Types.getTypes(namespace_id,query=f'{example_prefix}*'):
            print(f'type id: {type.Id}, type name: {type.Name}')
            if run:
                try:
                    ocsClient.Types.deleteType(namespace_id,type.Id)
                    ocsClient.Types.deleteType(namespace_id,type.TypeId)
                except:
                    pass
        
# cleanup - disabled, i.e: False
#cleanup(False)